### vae training

### imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

from models.VAE import VariationalAutoencoder
from utils.loaders import load_mnist

In [3]:
# run params
SECTION = 'vae'
RUN_ID = '0002'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #

### data

In [4]:
(x_train, y_train), (x_test, y_test) = load_mnist()

### architecture

In [5]:
vae = VariationalAutoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
    , r_loss_factor = 1000
)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [6]:
vae.encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 28, 28, 32)   0           encoder_conv_0[0][0]             
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 14, 14, 64)   18496       leaky_re_lu[0][0]                
____________________________________________________________________________________________

In [7]:
vae.decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 3136)              9408      
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 64)        0   

### training

In [8]:
LEARNING_RATE = 0.0005

In [9]:
vae.compile(LEARNING_RATE)

In [10]:
BATCH_SIZE = 32
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [11]:
vae.train(     
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

Epoch 1/200
1875/1875 [==============================] - ETA: 0s - loss: 58.5591 - reconstruction_loss: 55.3428 - kl_loss: 3.2163
Epoch 00001: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00001: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 118s 63ms/step - loss: 58.5594 - reconstruction_loss: 55.3427 - kl_loss: 3.2167 - lr: 5.0000e-04
Epoch 2/200
1875/1875 [==============================] - ETA: 0s - loss: 51.8337 - reconstruction_loss: 47.9265 - kl_loss: 3.9072
Epoch 00002: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00002: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 108s 58ms/step - loss: 51.8355 - reconstruction_loss: 47.9283 - kl_loss: 3.9072 - lr: 5.0000e-04
Epoch 3/200
1875/1875 [==============================] - ETA: 0s - loss: 50.3178 - reconstruction_loss: 46.1262 - kl_loss: 4.1916
Epoch 00003: saving model to run/vae/0002_digits\wei

1875/1875 [==============================] - ETA: 0s - loss: 45.7182 - reconstruction_loss: 40.5852 - kl_loss: 5.1330
Epoch 00020: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00020: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 123s 65ms/step - loss: 45.7178 - reconstruction_loss: 40.5848 - kl_loss: 5.1330 - lr: 5.0000e-04
Epoch 21/200
1875/1875 [==============================] - ETA: 0s - loss: 45.5680 - reconstruction_loss: 40.4270 - kl_loss: 5.1410
Epoch 00021: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00021: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 120s 64ms/step - loss: 45.5652 - reconstruction_loss: 40.4244 - kl_loss: 5.1409 - lr: 5.0000e-04
Epoch 22/200
1875/1875 [==============================] - ETA: 0s - loss: 45.5441 - reconstruction_loss: 40.3905 - kl_loss: 5.1536
Epoch 00022: saving model to run/vae/0002_digits\weights/weigh

1875/1875 [==============================] - ETA: 0s - loss: 44.7456 - reconstruction_loss: 39.4459 - kl_loss: 5.2997
Epoch 00039: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00039: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 120s 64ms/step - loss: 44.7478 - reconstruction_loss: 39.4482 - kl_loss: 5.2996 - lr: 5.0000e-04
Epoch 40/200
1875/1875 [==============================] - ETA: 0s - loss: 44.7470 - reconstruction_loss: 39.4332 - kl_loss: 5.3138- ETA:
Epoch 00040: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00040: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 120s 64ms/step - loss: 44.7463 - reconstruction_loss: 39.4325 - kl_loss: 5.3138 - lr: 5.0000e-04
Epoch 41/200
1875/1875 [==============================] - ETA: 0s - loss: 44.7040 - reconstruction_loss: 39.4003 - kl_loss: 5.3038
Epoch 00041: saving model to run/vae/0002_digits\weights

Epoch 59/200
1875/1875 [==============================] - ETA: 0s - loss: 44.2290 - reconstruction_loss: 38.8376 - kl_loss: 5.3914
Epoch 00059: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00059: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 110s 59ms/step - loss: 44.2275 - reconstruction_loss: 38.8363 - kl_loss: 5.3913 - lr: 5.0000e-04
Epoch 60/200
1875/1875 [==============================] - ETA: 0s - loss: 44.2230 - reconstruction_loss: 38.8324 - kl_loss: 5.3906
Epoch 00060: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00060: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 109s 58ms/step - loss: 44.2234 - reconstruction_loss: 38.8327 - kl_loss: 5.3907 - lr: 5.0000e-04
Epoch 61/200
1875/1875 [==============================] - ETA: 0s - loss: 44.2037 - reconstruction_loss: 38.8080 - kl_loss: 5.3958
Epoch 00061: saving model to run/vae/0002_digits\

1875/1875 [==============================] - ETA: 0s - loss: 43.8821 - reconstruction_loss: 38.4315 - kl_loss: 5.4506
Epoch 00078: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00078: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 107s 57ms/step - loss: 43.8840 - reconstruction_loss: 38.4335 - kl_loss: 5.4505 - lr: 5.0000e-04
Epoch 79/200
1875/1875 [==============================] - ETA: 0s - loss: 43.9217 - reconstruction_loss: 38.4687 - kl_loss: 5.4530
Epoch 00079: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00079: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 109s 58ms/step - loss: 43.9233 - reconstruction_loss: 38.4703 - kl_loss: 5.4530 - lr: 5.0000e-04
Epoch 80/200
1875/1875 [==============================] - ETA: 0s - loss: 43.9020 - reconstruction_loss: 38.4425 - kl_loss: 5.4595
Epoch 00080: saving model to run/vae/0002_digits\weights/weigh

...

1875/1875 [==============================] - 109s 58ms/step - loss: 43.5012 - reconstruction_loss: 37.9632 - kl_loss: 5.5379 - lr: 5.0000e-04
Epoch 118/200
1875/1875 [==============================] - ETA: 0s - loss: 43.4917 - reconstruction_loss: 37.9550 - kl_loss: 5.5367- ETA: 2s - loss: 43.4763 -
Epoch 00118: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00118: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 109s 58ms/step - loss: 43.4916 - reconstruction_loss: 37.9548 - kl_loss: 5.5368 - lr: 5.0000e-04
Epoch 119/200
1875/1875 [==============================] - ETA: 0s - loss: 43.4911 - reconstruction_loss: 37.9519 - kl_loss: 5.5392
Epoch 00119: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00119: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 109s 58ms/step - loss: 43.4951 - reconstruction_loss: 37.9557 - kl_loss: 5.5394 - lr: 5.0000e-04
Epoch 120

1875/1875 [==============================] - ETA: 0s - loss: 43.3407 - reconstruction_loss: 37.7842 - kl_loss: 5.5564
Epoch 00137: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00137: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 108s 58ms/step - loss: 43.3388 - reconstruction_loss: 37.7823 - kl_loss: 5.5565 - lr: 5.0000e-04
Epoch 138/200
1875/1875 [==============================] - ETA: 0s - loss: 43.3400 - reconstruction_loss: 37.7775 - kl_loss: 5.5624
Epoch 00138: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00138: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 107s 57ms/step - loss: 43.3388 - reconstruction_loss: 37.7764 - kl_loss: 5.5624 - lr: 5.0000e-04
Epoch 139/200
1875/1875 [==============================] - ETA: 0s - loss: 43.3348 - reconstruction_loss: 37.7597 - kl_loss: 5.5752
Epoch 00139: saving model to run/vae/0002_digits\weights/wei

Epoch 157/200
1875/1875 [==============================] - ETA: 0s - loss: 43.2481 - reconstruction_loss: 37.6603 - kl_loss: 5.5878
Epoch 00157: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00157: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 107s 57ms/step - loss: 43.2487 - reconstruction_loss: 37.6611 - kl_loss: 5.5876 - lr: 5.0000e-04
Epoch 158/200
1875/1875 [==============================] - ETA: 0s - loss: 43.2013 - reconstruction_loss: 37.6134 - kl_loss: 5.5879
Epoch 00158: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00158: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 107s 57ms/step - loss: 43.2008 - reconstruction_loss: 37.6130 - kl_loss: 5.5877 - lr: 5.0000e-04
Epoch 159/200
1875/1875 [==============================] - ETA: 0s - loss: 43.2203 - reconstruction_loss: 37.6328 - kl_loss: 5.5875
Epoch 00159: saving model to run/vae/0002_digi

Epoch 177/200
1875/1875 [==============================] - ETA: 0s - loss: 43.1293 - reconstruction_loss: 37.5283 - kl_loss: 5.6011
Epoch 00177: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00177: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 126s 67ms/step - loss: 43.1270 - reconstruction_loss: 37.5261 - kl_loss: 5.6010 - lr: 5.0000e-04
Epoch 178/200
1875/1875 [==============================] - ETA: 0s - loss: 43.1282 - reconstruction_loss: 37.5017 - kl_loss: 5.6265
Epoch 00178: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00178: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 125s 67ms/step - loss: 43.1295 - reconstruction_loss: 37.5031 - kl_loss: 5.6264 - lr: 5.0000e-04
Epoch 179/200
1875/1875 [==============================] - ETA: 0s - loss: 43.1074 - reconstruction_loss: 37.5020 - kl_loss: 5.6054
Epoch 00179: saving model to run/vae/0002_digi

Epoch 197/200
1875/1875 [==============================] - ETA: 0s - loss: 43.0711 - reconstruction_loss: 37.4454 - kl_loss: 5.6257
Epoch 00197: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00197: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 124s 66ms/step - loss: 43.0723 - reconstruction_loss: 37.4467 - kl_loss: 5.6256 - lr: 5.0000e-04
Epoch 198/200
1875/1875 [==============================] - ETA: 0s - loss: 42.9865 - reconstruction_loss: 37.3740 - kl_loss: 5.6125
Epoch 00198: saving model to run/vae/0002_digits\weights/weights.h5

Epoch 00198: saving model to run/vae/0002_digits\weights/weights.h5
1875/1875 [==============================] - 126s 67ms/step - loss: 42.9871 - reconstruction_loss: 37.3746 - kl_loss: 5.6125 - lr: 5.0000e-04
Epoch 199/200
1875/1875 [==============================] - ETA: 0s - loss: 43.0251 - reconstruction_loss: 37.4085 - kl_loss: 5.6166
Epoch 00199: saving model to run/vae/0002_digi